# Train

<https://github.com/ConnorJL/GPT2>

尝试训练 OpenWebText 的 En corpus

## 准备

要使用的 GPU:

In [1]:
%env CUDA_VISIBLE_DEVICES 2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
from IPython.display import display, Image
import sys

print(sys.version)

PYTHON = sys.executable
print(PYTHON)
PIP = f'{PYTHON} -m pip'

!{PYTHON} -V
%pip -V

3.6.7 | packaged by conda-forge | (default, Jul  2 2019, 02:18:42) 
[GCC 7.3.0]
/home/liuxy/miniconda3/envs/connorjl-gpt2/bin/python
Python 3.6.7
pip 19.1.1 from /home/liuxy/miniconda3/envs/connorjl-gpt2/lib/python3.6/site-packages/pip (python 3.6)


安装 tensorflow, SentencePiece，安装完毕后需要重启 Kernel:

In [2]:
%pip install -U tensorflow-gpu sentencepiece

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Requirement already up-to-date: sentencepiece in /home/liuxy/miniconda3/envs/connorjl-gpt2/lib/python3.6/site-packages (0.1.82)


## Pre-Process

语料文本转换为 tf 记录

语料文本在： `/home/Public/gpt2/openwebtext/1k/*.txt`

In [3]:
import os
from glob import glob, iglob

corpus_files_pattern = os.path.abspath('../../openwebtext/1k/*.txt')

In [4]:
!ls {corpus_files_pattern}

/home/Public/gpt2/openwebtext/1k/train_0.txt
/home/Public/gpt2/openwebtext/1k/train_100.txt
/home/Public/gpt2/openwebtext/1k/train_101.txt
/home/Public/gpt2/openwebtext/1k/train_102.txt
/home/Public/gpt2/openwebtext/1k/train_103.txt
/home/Public/gpt2/openwebtext/1k/train_104.txt
/home/Public/gpt2/openwebtext/1k/train_105.txt
/home/Public/gpt2/openwebtext/1k/train_106.txt
/home/Public/gpt2/openwebtext/1k/train_107.txt
/home/Public/gpt2/openwebtext/1k/train_108.txt
/home/Public/gpt2/openwebtext/1k/train_109.txt
/home/Public/gpt2/openwebtext/1k/train_10.txt
/home/Public/gpt2/openwebtext/1k/train_110.txt
/home/Public/gpt2/openwebtext/1k/train_111.txt
/home/Public/gpt2/openwebtext/1k/train_112.txt
/home/Public/gpt2/openwebtext/1k/train_113.txt
/home/Public/gpt2/openwebtext/1k/train_114.txt
/home/Public/gpt2/openwebtext/1k/train_115.txt
/home/Public/gpt2/openwebtext/1k/train_116.txt
/home/Public/gpt2/openwebtext/1k/train_117.txt
/home/Public/gpt2/openwebtext/1k/train_118.txt
/home/Public/gpt

### 生成 BPM 模型

使用 [google sentencepiece](https://github.com/google/sentencepiece)

In [5]:
# 使用 SPM 训练

input_str = ','.join(fn for fn in iglob(corpus_files_pattern))
!spm_train --input={input_str} --model_type=bpe --model_prefix=openwebtext --vocab_size=16000

sentencepiece_trainer.cc(49) LOG(INFO) Starts training with : 
TrainerSpec {
  input: /home/Public/gpt2/openwebtext/1k/train_190.txt
  input: /home/Public/gpt2/openwebtext/1k/train_206.txt
  input: /home/Public/gpt2/openwebtext/1k/train_221.txt
  input: /home/Public/gpt2/openwebtext/1k/train_166.txt
  input: /home/Public/gpt2/openwebtext/1k/train_75.txt
  input: /home/Public/gpt2/openwebtext/1k/train_234.txt
  input: /home/Public/gpt2/openwebtext/1k/train_24.txt
  input: /home/Public/gpt2/openwebtext/1k/train_215.txt
  input: /home/Public/gpt2/openwebtext/1k/train_126.txt
  input: /home/Public/gpt2/openwebtext/1k/train_112.txt
  input: /home/Public/gpt2/openwebtext/1k/train_150.txt
  input: /home/Public/gpt2/openwebtext/1k/train_148.txt
  input: /home/Public/gpt2/openwebtext/1k/train_192.txt
  input: /home/Public/gpt2/openwebtext/1k/train_246.txt
  input: /home/Public/gpt2/openwebtext/1k/train_76.txt
  input: /home/Public/gpt2/openwebtext/1k/train_39.txt
  input: /home/Public/gpt2/open

输出文件是:

In [6]:
!ls openwebtext.*

openwebtext.model  openwebtext.vocab


将他们复制到 `out` 目录

In [7]:
!cp -a  openwebtext.* ../out/openwebtext

### 生成 *.tfrecords

目录:

In [4]:
import os

tfrecords_dir = os.path.realpath('../out/openwebtext/tfrecords')
logs_dir = os.path.realpath('../logs')
bpe_model = os.path.realpath('../out/openwebtext/openwebtext.model')

我修改过了 `create_trfrecords.py`，用它生成！

In [9]:
!rm -rf {tfrecords_dir}/*.*
!rm -rf {logs_dir}/*.*

%cd ../datasets/openwebtext
try:
    !{PYTHON} create_tfrecords.py \
        -i "{corpus_files_pattern}" \
        -o {tfrecords_dir} \
        -n train -p 5  \
        -m {bpe_model} \
        -l {logs_dir} -s 16
finally:
    %cd ../../notebooks


/home/Public/gpt2/ConnorJL-GPT2/datasets/openwebtext
Got 250 files, divided into 50 chunks.
  0%|                                                    | 0/50 [00:00<?, ?it/s]WARNING: Logging before flag parsing goes to stderr.
W0716 13:17:35.856924 140409915266816 deprecation_wrapper.py:119] From create_tfrecords.py:75: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0716 13:17:35.857446 140409915266816 deprecation_wrapper.py:119] From create_tfrecords.py:75: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0716 13:17:35.857539 140409915266816 deprecation_wrapper.py:119] From create_tfrecords.py:75: The name tf.python_io.TFRecordWriter is deprecated. Please use tf.io.TFRecordWriter instead.

W0716 13:17:35.858688 140409915266816 deprecation_wrapper.py:119] From create_tfrecords.py:79: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.
W0716 13:17:35.858685 140409915266816

## Train

先看看 tf 的版本

In [5]:
import tensorflow as tf

print('tensorflow version: {}'.format(tf.__version__))
print('is_gpu_available = {}'.format(tf.test.is_gpu_available()))

tensorflow version: 1.14.0
is_gpu_available = True


### Configure file

In [5]:
import json

import sys
import os

json_cfg_file = "myopenwebtext.json"

train_conf = {
    "n_head": 12,
    "encoder_path": os.path.realpath("../out/openwebtext/openwebtext.model"),
    "n_vocab": 50257,
    "embed_dropout": 0.1,
    "lr": 0.00025,
    "warmup_steps": 2000,
    "beta1": 0.9,
    "beta2": 0.98,
    "epsilon": 1e-9,
    "opt_name": "adam",
    "weight_decay": 0.01,
    "train_batch_size": 4,
    "attn_dropout": 0.1,
    "train_steps": 10000,
    "eval_steps": 10,
    "max_steps": 500000,
    "data_path": tfrecords_dir,
    "scale": 0.2886751345948129,
    "res_dropout": 0.1,
    "predict_batch_size": 1,
    "eval_batch_size": 16,
    "iterations": 500,
    "n_embd": 768,
    "input": "myopenwebtext", # 输入处理函数，定义在 `input.py`
    "model": "GPT2",
    "model_path": os.path.realpath("../models/openwebtext"),
    "n_ctx": 1024,
    "predict_path": os.path.realpath("../logs/predictions.txt"),
    "n_layer": 12,
}

with open(os.path.join('..', json_cfg_file), "w") as fp:
    json.dump(train_conf, fp)

### Run train

In [6]:
%cd ..
try:
    !{PYTHON} main.py --model {json_cfg_file}
finally:
    %cd notebooks

/home/Public/gpt2/ConnorJL-GPT2
W0716 15:55:59.285621 139739900778240 deprecation_wrapper.py:119] From /home/Public/gpt2/ConnorJL-GPT2/optimizers.py:100: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0716 15:55:59.322466 139739900778240 deprecation_wrapper.py:119] From main.py:54: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

{'n_head': 12, 'encoder_path': '/home/Public/gpt2/ConnorJL-GPT2/out/openwebtext/openwebtext.model', 'n_vocab': 50257, 'embed_dropout': 0.1, 'lr': 0.00025, 'warmup_steps': 2000, 'beta1': 0.9, 'beta2': 0.98, 'epsilon': 1e-09, 'opt_name': 'adam', 'weight_decay': 0.01, 'train_batch_size': 4, 'attn_dropout': 0.1, 'train_steps': 10000, 'eval_steps': 10, 'max_steps': 500000, 'data_path': '/home/Public/gpt2/ConnorJL-GPT2/out/openwebtext/tfrecords', 'scale': 0.2886751345948129, 'res_dropout': 0.1, 'predict_batch_size': 1, 'eval_batch_size': 16, 'iterations': 500, 'n_e

In [40]:
print(f'''
训练出来的模型在: {train_conf['model_path']}
''')

!ls {train_conf['model_path']}


训练出来的模型在: /home/Public/gpt2/ConnorJL-GPT2/models/openwebtext

checkpoint
eval
events.out.tfevents.1563242242.yiren-Precision-T7600
events.out.tfevents.1563249303.yiren-Precision-T7600
graph.pbtxt
model.ckpt-0.data-00000-of-00001
model.ckpt-0.index
model.ckpt-0.meta


## Estimate

进行一次文本生成

In [41]:
%cd ..
try:
    !{PYTHON} main.py --model {json_cfg_file} --top_k 40 --predict_text "Hello there! My name is"
finally:
    %cd notebooks


/home/Public/gpt2/ConnorJL-GPT2
W0716 12:15:21.344404 140404054505216 deprecation_wrapper.py:119] From /home/Public/gpt2/ConnorJL-GPT2/optimizers.py:100: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0716 12:15:21.421810 140404054505216 deprecation_wrapper.py:119] From main.py:54: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

{'n_head': 12, 'encoder_path': '/home/Public/gpt2/ConnorJL-GPT2/out/openwebtext/openwebtext.model', 'n_vocab': 50257, 'embed_dropout': 0.1, 'lr': 0.00025, 'warmup_steps': 2000, 'beta1': 0.9, 'beta2': 0.98, 'epsilon': 1e-09, 'opt_name': 'adam', 'weight_decay': 0.01, 'train_batch_size': 32, 'attn_dropout': 0.1, 'train_steps': 10000, 'eval_steps': 10, 'max_steps': 500000, 'data_path': '/home/Public/gpt2/ConnorJL-GPT2/out/openwebtext/tfrecords', 'scale': 0.2886751345948129, 'res_dropout': 0.1, 'predict_batch_size': 1, 'eval_batch_size': 32, 'iterations': 500, 'n_